#### Loading all the libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_digits


In [2]:
digits = load_digits()

In [3]:
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=3)

In [4]:
svm_model = svm.SVC(kernel='rbf', C=30, gamma = 'auto')
svm_model.fit(X_train,y_train)
svm_model.score(X_test,y_test)

0.3333333333333333

In [5]:
cross_val_score(svm.SVC(kernel='rbf', C=10, gamma='auto'), digits.data, digits.target,cv=5)

array([0.45277778, 0.46944444, 0.47910864, 0.47910864, 0.50139276])

In [6]:
cross_val_score(svm.SVC(kernel='linear', C=10, gamma='auto'), digits.data, digits.target,cv=5)

array([0.96388889, 0.91944444, 0.96657382, 0.9637883 , 0.92479109])

In [7]:
cross_val_score(svm.SVC(kernel='linear', C=50, gamma='auto'), digits.data, digits.target,cv=5)

array([0.96388889, 0.91944444, 0.96657382, 0.9637883 , 0.92479109])

In [8]:
kernels = ['rbf','linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval, C=cval, gamma='auto'), digits.data, digits.target, cv=5)
        avg_scores[kval+'_'+str(cval)]=np.average(cv_scores)

In [9]:
avg_scores

{'rbf_1': 0.448545341999381,
 'rbf_10': 0.47636645001547506,
 'rbf_20': 0.47636645001547506,
 'linear_1': 0.9476973073351903,
 'linear_10': 0.9476973073351903,
 'linear_20': 0.9476973073351903}

In [10]:
clf = GridSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel': ['rbf','linear']
    }, cv=5,return_train_score=False)
clf.fit(digits.data,digits.target)
df_clf_score=pd.DataFrame(clf.cv_results_)

In [11]:
df_clf_score[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.448545
1,1,linear,0.947697
2,10,rbf,0.476366
3,10,linear,0.947697
4,20,rbf,0.476366
5,20,linear,0.947697


In [12]:
clf.best_estimator_

SVC(C=1, gamma='auto', kernel='linear')

In [13]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [14]:
clfr = RandomizedSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear']
},cv=5,return_train_score=False,n_iter=2)
clfr.fit(digits.data,digits.target)
df_clfr_score=pd.DataFrame(clfr.cv_results_)

In [15]:
df_clfr_score[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.947697
1,20,linear,0.947697


### Gridsearch - All models

In [16]:
model_params = {
    'svm':{
        'model':svm.SVC(gamma='auto'),
        'params': {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }
    },
    'random_forest':{
    'model':RandomForestClassifier(),
    'params':{
        'n_estimators':[1,5,10,20,50]
    }
},
    'lr':{
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
        'params':{
            'C':[1,5,10]
        }
    },
    'naivebayesGaussian': {
        'model': GaussianNB(),
        'params':{}
    },
    'naivebayesMultinominal': {
        'model': MultinomialNB(),
        'params':{}
    },
    'decisionTree':{
        'model':DecisionTreeClassifier(),
        'params':{
            'criterion':['gini','entropy']
        }
    }
}

In [17]:
scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'],cv=5, return_train_score=False)
    clf.fit(digits.data, digits.target)
    scores.append({
        'model':model_name,
        'best_score':clf.best_score_,
        'best_params':clf.best_estimator_
    })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.947697,"SVC(C=1, gamma='auto', kernel='linear')"
1,random_forest,0.936028,"(DecisionTreeClassifier(max_features='auto', r..."
2,lr,0.922114,"LogisticRegression(C=1, solver='liblinear')"
3,naivebayesGaussian,0.806928,GaussianNB()
4,naivebayesMultinominal,0.870350,MultinomialNB()
5,decisionTree,0.810252,DecisionTreeClassifier(criterion='entropy')
